# Pulling Venue Data from FourSquare API

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Testing the Code](#Testing-the-Code)

[Making the DataFrame](#Making-the-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Venue and Distance](#Sorting-by-Venue-and-Distance)

[Obtaining Neighborhood for Each Arcade](#Obtaining-Neighborhood-for-Each-Arcade)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [5]:
seattle_df = pd.read_csv('seattle_geodata.csv')
seattle_df.head(10)

,Within larger district,Neighborhood name,area_longitude,area_latitude,hood_longitude,hood_latitude
0,Seattle,North Seattle,-122.3398649,47.6020234,-122.29145496064257,47.590054550000005
1,North Seattle,Broadview,-122.29145496064257,47.590054550000005,-122.3604067,47.7223199
2,North Seattle,Bitter Lake,-122.29145496064257,47.590054550000005,-122.3487639,47.7262358
3,North Seattle,North Beach,-122.29145496064257,47.590054550000005,-122.392362,47.6962103
4,North Seattle,Crown Hill,-122.29145496064257,47.590054550000005,-122.3714586,47.6947154
5,North Seattle,Greenwood,-122.29145496064257,47.590054550000005,-122.3548773,47.6909806
6,North Seattle,Northgate,-122.29145496064257,47.590054550000005,-122.3212307,47.7131534
7,Northgate,Haller Lake,-122.3212307,47.7131534,-122.33375132229858,47.7197478
8,Northgate,Pinehurst,-122.3212307,47.7131534,-122.31554228404718,47.712876550000004
9,Northgate,North College Park,-122.3212307,47.7131534,No Data Found,No Data Found


### Connecting to FourSquare

In [6]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 30

### Testing the Code

In [16]:
#assign variables for the search
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [17]:
for index, hood in enumerate(seattle_df['hood_latitude']):
    if index == 26: #preselected based on venues in initial search
        latitude= seattle_df['hood_latitude'][index]
        longitude= seattle_df['hood_longitude'][index]
print(latitude, longitude)

47.6612984 -122.3131523


In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '613a46937d5620736911c57f'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e4c7872bd413c4cc669a152',
    'name': "Paul's Italian Kitchen",
    'location': {'address': '4333 University Way NE',
     'lat': 47.660812,
     'lng': -122.313468,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.660812,
       'lng': -122.313468}],
     'distance': 59,
     'postalCode': '98105',
     'cc': 'US',
     'city': 'Seattle',
     'state': 'WA',
     'country': 'United States',
     'formattedAddress': ['4333 University Way NE', 'Seattle, WA 98105']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1631209107',
    'hasPer

In [25]:
ident=results['response']['venues'][0]
type(ident)
for dictionary in results['response']['venues']:
    print(dictionary['id'])
    print(dictionary['name'])
    print(dictionary['location']['address'])
    print(dictionary['location']['lat'])
    print(dictionary['location']['lng'])
    print(dictionary['location']['distance'])
    for cat in dictionary['categories']:
        print(cat['id'], cat['name'])

4e4c7872bd413c4cc669a152
Paul's Italian Kitchen
4333 University Way NE
47.660812
-122.313468
59
4bf58dd8d48988d110941735 Italian Restaurant
4f32450919836c91c7c703b4
Ave Ristorante Italiano
4743 University Way NE
47.664329528808594
-122.31327056884766
337
4d4b7105d754a06374d81259 Food


### Making the DataFrame

In [9]:
#assign variables for the search
search_query = 'Arcade'
radius = 2000
print(search_query + ' .... OK!')

Arcade .... OK!


In [10]:
seattle_venues = pd.DataFrame(columns = ['Hood','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
seattle_venues.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [13]:
for index, hood in enumerate(seattle_df['hood_latitude']):
    try:
        hood_name = seattle_df['Neighborhood name'][index]
        latitude = seattle_df['hood_latitude'][index]
        longitude = seattle_df['hood_longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
                continue
        seattle_venues = seattle_venues.append({'Venue_Distance':ven_dist,'Hood':hood_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)

In [14]:
seattle_venues = seattle_venues.set_index('Hood')
print(len(seattle_venues))
seattle_venues

255


,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
Hood,,,,,,,,
Olympic Hills,578cabc2498e2cf4c49feb3d,ShopsArcade,1598,15218 12th Ave NE,47.739429,-122.315063,4bf58dd8d48988d1f6941735,Department Store
Roosevelt,4af30888f964a520f5e921e3,4518 Arcade,1722,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
Ravenna,4af30888f964a520f5e921e3,4518 Arcade,1927,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
University Village,4af30888f964a520f5e921e3,4518 Arcade,1069,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
Wallingford,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,1740,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade
Wallingford,4af30888f964a520f5e921e3,4518 Arcade,1612,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
Northlake,4af30888f964a520f5e921e3,4518 Arcade,1180,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade


In [15]:
print(len(seattle_venues))
seattle_venues.to_csv('{}_venue_data.csv'.format(search_query))

255


### Dropping the Nulls

In [16]:
seattle_venues = pd.read_csv('{}_venue_data.csv'.format(search_query))
print(search_query)
seattle_venues.head()

Arcade


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Olympic Hills,578cabc2498e2cf4c49feb3d,ShopsArcade,1598,15218 12th Ave NE,47.739429,-122.315063,4bf58dd8d48988d1f6941735,Department Store
1,Roosevelt,4af30888f964a520f5e921e3,4518 Arcade,1722,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
2,Ravenna,4af30888f964a520f5e921e3,4518 Arcade,1927,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
3,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
4,University Village,4af30888f964a520f5e921e3,4518 Arcade,1069,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office


In [17]:
print(len(seattle_venues))

255


In [18]:
seattle_venues = seattle_venues[seattle_venues != 'Null']
seattle_venues = seattle_venues[seattle_venues != '[]']
seattle_venues = seattle_venues.dropna()
print(len(seattle_venues))
seattle_venues = seattle_venues.reset_index()
seattle_venues = seattle_venues.drop(['index'], axis = 1)
seattle_venues.head()

255


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Olympic Hills,578cabc2498e2cf4c49feb3d,ShopsArcade,1598,15218 12th Ave NE,47.739429,-122.315063,4bf58dd8d48988d1f6941735,Department Store
1,Roosevelt,4af30888f964a520f5e921e3,4518 Arcade,1722,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
2,Ravenna,4af30888f964a520f5e921e3,4518 Arcade,1927,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
3,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
4,University Village,4af30888f964a520f5e921e3,4518 Arcade,1069,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office


### Sorting by Venue and Distance

In [19]:
seattle_sorted = seattle_venues.sort_values(['Venue_ID','Venue_Distance'], ascending = True)
seattle_sorted.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
3,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
61,Stevens,4af30888f964a520f5e921e3,4518 Arcade,860,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
4,University Village,4af30888f964a520f5e921e3,4518 Arcade,1069,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
7,Northlake,4af30888f964a520f5e921e3,4518 Arcade,1180,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
204,Central Area,4af30888f964a520f5e921e3,4518 Arcade,1517,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office


In [20]:
seattle_sorted['Category_Name'].unique()

array(['Office', 'Smoke Shop', 'Arcade', 'Strip Club', 'Shopping Plaza',
       'Food', 'Other Nightlife', 'Department Store',
       'Fast Food Restaurant'], dtype=object)

In [21]:
seattle_sorted = seattle_sorted[seattle_sorted['Category_Name'] != 'Smoke Shop']
seattle_sorted = seattle_sorted[seattle_sorted['Category_Name'] != 'Strip Club']
seattle_sorted = seattle_sorted[seattle_sorted['Category_Name'] != 'Fast Food Restaurant']
seattle_sorted = seattle_sorted[seattle_sorted['Category_Name'] != 'Food']
#seattle_sorted = seattle_sorted[seattle_sorted['Category_Name'] != 'Other Nightlife']
seattle_sorted['Category_Name'].unique()

array(['Office', 'Arcade', 'Shopping Plaza', 'Other Nightlife',
       'Department Store'], dtype=object)

In [22]:
seattle_sorted.to_csv('{}_sorted.csv'.format(search_query))

### Obtaining Neighborhood for Each Arcade

In [23]:
seattle_sorted = pd.read_csv('{}_sorted.csv'.format(search_query))

In [24]:
seattle_sorted = seattle_sorted.drop('Unnamed: 0',axis =1 )

In [25]:
print(len(seattle_sorted))
seattle_sorted

177


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
1,Stevens,4af30888f964a520f5e921e3,4518 Arcade,860,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
2,University Village,4af30888f964a520f5e921e3,4518 Arcade,1069,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
3,Northlake,4af30888f964a520f5e921e3,4518 Arcade,1180,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
4,Central Area,4af30888f964a520f5e921e3,4518 Arcade,1517,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
5,Wallingford,4af30888f964a520f5e921e3,4518 Arcade,1612,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
6,Roosevelt,4af30888f964a520f5e921e3,4518 Arcade,1722,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
7,Portage Bay,4af30888f964a520f5e921e3,4518 Arcade,1859,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
8,Ravenna,4af30888f964a520f5e921e3,4518 Arcade,1927,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
9,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade


In [26]:
count = -1
list_to_drop = list()
for ID in seattle_sorted['Venue_ID']:
    count = count +1
    try:
        if seattle_sorted['Venue_ID'][count] == seattle_sorted['Venue_ID'][count-1] and seattle_sorted['Venue_Distance'][count]>=seattle_sorted['Venue_Distance'][count-1]:
            list_to_drop.append(count)
    except:
        continue
list_to_drop

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 86,
 87,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 175,
 176]

In [27]:
seattle_final = seattle_sorted.drop(index = list_to_drop)

In [28]:
print(len(seattle_final))
seattle_final

15


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
9,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade
20,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade
27,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza
42,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade
52,Pioneer Square,4fb7f757e4b09a1e8cab118e,Time masheen arcade,252,No Address,47.601545,-122.330684,4bf58dd8d48988d1e1931735,Arcade
70,Lower Queen Anne,4fdd404be4b082d0767d2155,Seattle Interactive Media Museum,387,No Address,47.621242,-122.350853,4bf58dd8d48988d1e1931735,Arcade
85,Renton Hill,4ff503b1e4b02ca33b4b731f,Rod's Pinball Arcade,829,9818 Beacon Ave S,47.512992,-122.272850,4bf58dd8d48988d1e1931735,Arcade
88,Lower Queen Anne,5064c506e4b06062642548ae,Next 50 Video Arcade,390,Seattle Center,47.621220,-122.350827,4bf58dd8d48988d1e1931735,Arcade
103,Harrison,50fa31bae4b020b5559a65cc,Lipton's Arcade,373,No Address,47.618770,-122.326777,4bf58dd8d48988d11a941735,Other Nightlife


In [29]:
seattle_final.to_csv('{}_unique.csv'.format(search_query))

### Pulling the Ratings

In [30]:
search_query

'Arcade'

In [39]:
seattle_final = pd.read_csv('{}_unique.csv'.format(search_query))
seattle_final.head()

,Unnamed: 0,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
1,9,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade
2,20,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade
3,27,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza
4,42,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade


In [40]:
seattle_final = seattle_final.drop('Unnamed: 0', axis = 1)
seattle_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office
1,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade
2,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade
3,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza
4,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade


In [37]:
seattle_final = seattle_final.assign(Venue_Rating='NaN')
seattle_final

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office,NaN
1,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade,NaN
2,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade,NaN
3,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza,NaN
4,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade,NaN
5,Pioneer Square,4fb7f757e4b09a1e8cab118e,Time masheen arcade,252,No Address,47.601545,-122.330684,4bf58dd8d48988d1e1931735,Arcade,NaN
6,Lower Queen Anne,4fdd404be4b082d0767d2155,Seattle Interactive Media Museum,387,No Address,47.621242,-122.350853,4bf58dd8d48988d1e1931735,Arcade,NaN
7,Renton Hill,4ff503b1e4b02ca33b4b731f,Rod's Pinball Arcade,829,9818 Beacon Ave S,47.512992,-122.272850,4bf58dd8d48988d1e1931735,Arcade,NaN
8,Lower Queen Anne,5064c506e4b06062642548ae,Next 50 Video Arcade,390,Seattle Center,47.621220,-122.350827,4bf58dd8d48988d1e1931735,Arcade,NaN
9,Harrison,50fa31bae4b020b5559a65cc,Lipton's Arcade,373,No Address,47.618770,-122.326777,4bf58dd8d48988d11a941735,Other Nightlife,NaN


In [42]:
count = -1
rating_list = list()
for ident in seattle_final['Venue_ID']:
    count = count +1
    venue_id = seattle_final['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 8.8,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated']

In [43]:
seattle_final = seattle_final.assign(Venue_Rating = rating_list)

In [44]:
seattle_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office,Not Rated
1,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade,Not Rated
2,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza,Not Rated
4,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade,8.8


In [45]:
seattle_final.to_csv('{}_final_clean.csv'.format(search_query))